In [244]:
import torch

In [245]:
batch_size = 64
input_size = 3
hidden_size = 2
output_size = 1

dtype = torch.float
device = torch.device("cpu")

In [246]:
# Create random input and output data
x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

In [247]:
# Randomly initialize weights
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype)

#b = torch.randn(input_size,output_size, device=device, dtype=dtype)
b2 = torch.randn(batch_size,output_size, device=device, dtype=dtype)
b1 = torch.randn(batch_size,hidden_size, device=device, dtype=dtype)

In [248]:
# код в домашке реализован иначе, чем на лекции. мне как неподготовленному человеку пока сложно было уловить 
# суть на каждом шаге и что происходит в каждом нейроне, поэтому так. Неоптимально, обычно я так не делаю, 
# чисто чтобы разобраться
# Надеюсь на понимание :-)

import numpy as np
import torch

np_size = 64
input_size = 3
hidden_size = 2
output_size = 1

def d_relu(x):
    # Производная relu
    if x<= 0:
        return 0
    else:
        return 1

def relu(x):
    return max(0.0, x)

def mse_loss(y_true, y_pred): 
    return ((y_true - y_pred) ** 2).mean()

class nNetwork:
    '''
    Нейронная сеть с:
    - 3 входами
    - скрытым слоем с 2 нейронами (h1, h2)
    - выходным слоем с 1 нейроном (o1)
    '''

    def __init__(self):
        # Веса
        self.w1 = np.random.normal()
        self.w2 = np.random.normal()
        self.w3 = np.random.normal()
        self.w4 = np.random.normal()
        self.w5 = np.random.normal()
        self.w6 = np.random.normal()
        self.w7 = np.random.normal()
        self.w8 = np.random.normal()

        # Пороги
        self.b1 = np.random.normal()
        self.b2 = np.random.normal()
        self.b3 = np.random.normal()
        
        #Запомним веса для последующего обучения
        self.w_1 = [[self.w1,self.w2],[self.w3,self.w4],[self.w5,self.w6]]
        self.w_2 = [self.w7,self.w8]

    def feedforward(self, x):
        # x входные данные
        h1 = relu(self.w1 * x[0] + self.w3 * x[1] + self.w5 * x[2] + self.b1)
        h2 = relu(self.w2 * x[0] + self.w4 * x[1] + self.w6 * x[2] + self.b2)
        o1 = relu(self.w7 * h1 + self.w8 * h2 + self.b3)
        return o1

    def train(self, data, all_y_trues):
        learn_rate = 1e-6
        epochs = 500 # сколько раз пройти по всему набору данных 

        for epoch in range(epochs):
            for x, y_true in zip(data, all_y_trues):
                # --- Прямой проход
                sum_h1 = self.w1 * x[0] + self.w3 * x[1] + self.w5 * x[2] + self.b1
                h1 = relu(sum_h1)

                sum_h2 = self.w2 * x[0] + self.w4 * x[1] + self.w6 * x[2] + self.b2
                h2 = relu(sum_h2)

                sum_o1 = self.w7 * h1 + self.w8 * h2 + self.b3
                o1 = relu(sum_o1)
                y_pred = o1

                # Считаем частные производные:
                d_L_d_ypred = -2 * (y_true - y_pred)

                # Нейрон o1
                d_ypred_d_w7 = h1 * d_relu(sum_o1)
                d_ypred_d_w8 = h2 * d_relu(sum_o1)
                d_ypred_d_b3 = d_relu(sum_o1)

                d_ypred_d_h1 = self.w7 * d_relu(sum_o1)
                d_ypred_d_h2 = self.w8 * d_relu(sum_o1)

                # Нейрон h1
                d_h1_d_w1 = x[0] * d_relu(sum_h1)
                d_h1_d_w3 = x[1] * d_relu(sum_h1)
                d_h1_d_w5 = x[2] * d_relu(sum_h1)
                d_h1_d_b1 = d_relu(sum_h1)

                # Нейрон h2
                d_h2_d_w2 = x[0] * d_relu(sum_h2)
                d_h2_d_w4 = x[1] * d_relu(sum_h2)
                d_h2_d_w6 = x[2] * d_relu(sum_h2)
                d_h2_d_b2 = d_relu(sum_h2)

                # --- Обновляем веса и пороги
                # Нейрон h1
                self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
                self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w3
                self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w5
                self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1

                # Нейрон h2
                self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w2
                self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
                self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w6
                self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

                # Нейрон o1
                self.w7 -= learn_rate * d_L_d_ypred * d_ypred_d_w7
                self.w8 -= learn_rate * d_L_d_ypred * d_ypred_d_w8
                self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

            # --- Считаем полные потери в конце каждой эпохи
            if epoch % 100 == 0:
                y_preds = np.apply_along_axis(self.feedforward, 1, data)
                loss = mse_loss(all_y_trues, y_preds)
                print("Epoch %d loss: %.3f" % (epoch, loss))

# Определим набор данных
data = np.random.sample((np_size, input_size))
#x_t = torch.tensor(data)
all_y_trues = np.random.sample((np_size, 1))
#y_t = torch.tensor(all_y_trues)

# Обучаем
network = nNetwork()
network.train(data, all_y_trues)

Epoch 0 loss: 0.297
Epoch 100 loss: 0.297
Epoch 200 loss: 0.297
Epoch 300 loss: 0.297
Epoch 400 loss: 0.297


In [257]:
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
    
    # Теперь подсчет градиентов для весов происходит при вызове backward
    loss.backward()
   
    # Обновляем значение весов, но укзаываем, чтобы PyTorch не считал эту операцию, 
    # которая бы учавствовала бы при подсчете градиентов в chain rule
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # Теперь обнуляем значение градиентов, чтобы на следующем шаге 
        # они не учитывались при подсчете новых градиентов,
        # иначе произойдет суммирвоание старых и новых градиентов
        w1.grad.zero_()
        w2.grad.zero_()

99 172.85443115234375
199 164.02105712890625
299 155.9771270751953
399 148.6368865966797
499 141.9258575439453


In [256]:
import torch.optim as optim

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
optimizer = torch.optim.Adam([w1, w2, b1, b2], lr=learning_rate)

for t in range(500):
    optimizer.zero_grad()
    
    y_pred = (x.mm(w1).clamp(min=0) + b1).mm(w2) + b2
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    loss.backward()
   
    optimizer.step()

99 1350.751220703125
199 1350.484619140625
299 1350.218017578125
399 1349.95166015625
499 1349.685302734375
